In [1]:
import csv
all_path_list=[]
all_labels=[]
lc=0
with open("D:/Test_Disen/Backend/coco_train.csv") as f:
    csvreader = csv.reader(f)
    for line in csvreader:
        if lc>0:
            all_path_list.append(line[1])
            all_labels.append(line[2].split(','))
        lc=lc+1

In [2]:
from pycocotools.coco import COCO
json_path = "D:/coco2017/annotations/instances_train2017.json"
img_path = "D:/coco2017/train2017"
coco = COCO(annotation_file=json_path)
coco_classes = dict([(v["id"], v["name"]) for k, v in coco.cats.items()])
all_cats=list(coco_classes.values())

loading annotations into memory...
Done (t=16.76s)
creating index...
index created!


In [3]:
# import numpy as np
# from sklearn.neighbors import BallTree
# retrieve_img_npy=np.load("D:/Test_Disen/Backend/train2017.npy")
# retrieve_ball=BallTree(retrieve_img_npy, metric='euclidean')

In [4]:
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from audtorch.metrics.functional import pearsonr
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
# from torchvision import datasets
from torchvision.transforms import ToTensor
import cv2
from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA

In [103]:
class image_text_dataset(Dataset):
    def __init__(self, list_image_path, list_ref_path1, list_ref_path2, class_1, class_2):

        self.image_path = list_image_path
        self.ref_image_path1 = list_ref_path1
        self.ref_image_path2 = list_ref_path2
        self.class_1 = clip.tokenize(class_1)
        self.class_2 = clip.tokenize(class_2)
        # self.labs  = list_lab # you can tokenize everything at once in here(slow at the beginning), or tokenize it in the training loop.

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, idx):
        image = preprocess(Image.open(self.image_path[idx])) # Image from PIL module
        ref_imgs1 = [preprocess(Image.open(v)) for v in self.ref_image_path1]
        ref_imgs2 = [preprocess(Image.open(v)) for v in self.ref_image_path2]
        lab1 = self.class_1
        lab2 = self.class_2
        return image, ref_imgs1, ref_imgs2, lab1, lab2

class image_text_dataset1(Dataset):
    def __init__(self, list_image_path, list_ref_path1, list_ref_path2, class_1):
        self.image_path = list_image_path
        self.ref_image_path1 = list_ref_path1
        self.ref_image_path2 = list_ref_path2
        self.class_1 = clip.tokenize(class_1)
        # self.class_1 = class_1
        # self.labs  = list_lab # you can tokenize everything at once in here(slow at the beginning), or tokenize it in the training loop.

    def __len__(self):
        # return len(self.class_1)
        return len(self.image_path)

    def __getitem__(self, idx):
        image = preprocess(Image.open(self.image_path[idx]))
        ref_imgs1 = [preprocess(Image.open(v)) for v in self.ref_image_path1]
        ref_imgs2 = [preprocess(Image.open(v)) for v in self.ref_image_path2]
        # lab1 = clip.tokenize(self.class_1[0])#self.class_1
        lab1 = self.class_1
        return image, ref_imgs1, ref_imgs2, lab1

def rank_loss(img_high, txt_high, img_low, txt_low, num):
    matrix=torch.norm(txt_low-img_low, dim=1).repeat(num,1)
    # dis=torch.norm(matrix - matrix.transpose(0, 1), dim=-1)
    dis=matrix-matrix.T
    hmatrix=torch.norm(txt_high-img_high,dim=1).repeat(num,1)
    hdis=hmatrix-hmatrix.T
    return nn.ReLU()(-dis*hdis)/(torch.norm(txt_low-img_low)+0.0001)

def rank_loss1(img_high, txt_high, img_low, txt_low, num):
    # num=150
    # num=sample_len
#     num=len(generated_paths[0])
    matrix=torch.norm(txt_low-img_low, dim=1).repeat(num,1)
    # dis=torch.norm(matrix - matrix.transpose(0, 1), dim=-1)
    dis=matrix-matrix.T
    hmatrix=torch.norm(txt_high-img_high,dim=1).repeat(num,1)

    hdis=hmatrix-hmatrix.T
#     return nn.ReLU()(-dis*hdis)/(torch.norm(txt_low-img_low)+0.0001)
    return nn.ReLU()(-num*dis*hdis*torch.exp(-0.1*torch.minimum(hmatrix, hmatrix.T)**2)/torch.sum(torch.exp(-0.1*hmatrix**2)))/(torch.norm(txt_low-img_low)+0.0001)





class image_textanchor_dataset(Dataset):
    def __init__(self, image_features, text_features):
        self.image_features = image_features
        self.text_features = text_features
        # self.KNNS = KNNS
    def __len__(self):
        return 1

    def __getitem__(self, idx):
        image_tensor=torch.Tensor(self.image_features)
        text_tensor=torch.Tensor(self.text_features)
        # anchor_tensor=torch.IntTensor(self.KNNS)
        return image_tensor, text_tensor

class image_textanchor_dataset1(Dataset):
    def __init__(self, image_features, text_features, multiply, bsize):
        self.image_features = image_features
        self.text_features = text_features
        # self.KNNS = KNNS
    def __len__(self):
        return multiply

    def __getitem__(self, idx):
        image_tensor=torch.Tensor(self.image_features[bsize*idx:bsize*(idx+1),:])
        text_tensor=torch.Tensor(self.text_features)
        # anchor_tensor=torch.IntTensor(self.KNNS)
        return image_tensor, text_tensor

class NeuralProj(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            # nn.Linear(1024, 512),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 2),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    
class NeuralProj_MMD(nn.Module):
    def __init__(self):
        super().__init__()
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(1024, 1024),
# #             nn.ReLU(),
# # #             nn.Linear(512, 256),
# # #             nn.ReLU(),
# #             nn.Linear(512, 1024),
# #             nn.ReLU(),
#         )
        self.linear_relu_stack = nn.Sequential(
#             nn.Linear(1024, 512),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 2),
        )

#     def forward(self, x):
#         logits0 = self.linear_relu_stack(x)
#         logits = self.linear_relu_stack1(logits0)
#         return logits0, logits
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits, logits

# ProjTextModel = NeuralProj().to(device)
# ProjImgModel = NeuralProj().to(device)

def data_context_map(img_embedding, txt_embedding, img_projection, txt_projection):
    img0=img_embedding
    txt0=txt_embedding
    img_embedding=torch.flatten(img_embedding, start_dim=1)
    num = img_embedding.shape[0]
    img_matrix = img_embedding.repeat(num, 1, 1)
    txt_embedding=torch.flatten(txt_embedding, start_dim=1)
    num1 = txt_embedding.shape[0]
    txt_matrix = txt_embedding.repeat(num1, 1, 1)
    txt_matrix0 = txt_embedding.repeat(num, 1, 1)
    img_matrix0 = img_embedding.repeat(num1, 1, 1)
    img_dis = torch.norm(img_matrix - img_matrix.transpose(0, 1), dim=-1)
    txt_dis = torch.norm(txt_matrix - txt_matrix.transpose(0, 1), dim=-1)
    cross_dis = torch.norm(txt_matrix0 - img_matrix0.transpose(0, 1), dim=-1)
#     cross_dis=1-torch.matmul(img0,txt0.T)
    img_dis=img_dis/torch.mean(img_dis)
    txt_dis=txt_dis/torch.mean(txt_dis)
    cross_dis=cross_dis/torch.mean(cross_dis)
    merge_dis0=torch.cat((img_dis, cross_dis),dim=1)
    merge_dis1=torch.cat((cross_dis.T, txt_dis),dim=1)
    merge_dis=torch.cat((merge_dis0, merge_dis1),dim=0)
    low_img0=img_projection
    low_txt0=txt_projection
    low_merge=torch.cat((img_projection, txt_projection), dim=0)
    # print(low_merge.shape)
    img_projection=torch.flatten(img_projection, start_dim=1)
    num = img_projection.shape[0]
    low_img_matrix = img_projection.repeat(num, 1, 1)
    txt_projection=torch.flatten(txt_projection, start_dim=1)
    num1 = txt_projection.shape[0]
    low_txt_matrix = txt_projection.repeat(num1, 1, 1)
    low_txt_matrix0 = txt_projection.repeat(num, 1, 1)
    low_img_matrix0 = img_projection.repeat(num1, 1, 1)
    low_img_dis = torch.norm(low_img_matrix - low_img_matrix.transpose(0, 1), dim=-1)
    low_txt_dis = torch.norm(low_txt_matrix - low_txt_matrix.transpose(0, 1), dim=-1)
    low_merge=torch.flatten(low_merge, start_dim=1)
    num2=low_merge.shape[0]
    low_merge_matrix=low_merge.repeat(num2, 1, 1)
    low_merge_dis = torch.norm(low_merge_matrix - low_merge_matrix.transpose(0, 1), dim=-1) 
    low_cross_dis = torch.norm(low_txt_matrix0 - low_img_matrix0.transpose(0, 1), dim=-1)
    img_corr = torch.mean(
    pearsonr(
        img_dis,
        low_img_dis,
    )[:, 0])
    txt_corr = torch.mean(
    pearsonr(
        txt_dis,
        low_txt_dis,
    )[:, 0])
    cross_corr = torch.mean(
    pearsonr(
        cross_dis,
        low_cross_dis,
    )[:, 0])
    all_corr= torch.mean(
    pearsonr(
        merge_dis,
        low_merge_dis,
    )[:, 0])
    return -img_corr, -txt_corr, -cross_corr, -all_corr, merge_dis, img_dis, txt_dis, cross_dis, low_merge_dis, low_img_dis, low_txt_dis, low_cross_dis

def data_context_map1(img_embedding, txt_embedding, img_projection, txt_projection):
    img0=img_embedding
    txt0=txt_embedding
    img_embedding=torch.flatten(img_embedding, start_dim=1)
    num = img_embedding.shape[0]
    img_matrix = img_embedding.repeat(num, 1, 1)
    txt_embedding=torch.flatten(txt_embedding, start_dim=1)
    num1 = txt_embedding.shape[0]
    txt_matrix = txt_embedding.repeat(num1, 1, 1)
    txt_matrix0 = txt_embedding.repeat(num, 1, 1)
    img_matrix0 = img_embedding.repeat(num1, 1, 1)
    img_dis = torch.norm(img_matrix - img_matrix.transpose(0, 1), dim=-1)
    txt_dis = torch.norm(txt_matrix - txt_matrix.transpose(0, 1), dim=-1)
    cross_dis = torch.norm(txt_matrix0 - img_matrix0.transpose(0, 1), dim=-1)
#     cross_dis=1-torch.matmul(img0,txt0.T)
    img_dis=img_dis/torch.mean(img_dis)
    txt_dis=txt_dis/torch.mean(txt_dis)
    cross_dis=cross_dis/torch.mean(cross_dis)
#     print(cross_dis.shape)
    merge_dis0=torch.cat((img_dis, cross_dis),dim=1)
    merge_dis1=torch.cat((cross_dis.T, txt_dis),dim=1)
    merge_dis=torch.cat((merge_dis0, merge_dis1),dim=0)
    low_img0=img_projection
    low_txt0=txt_projection
    low_merge=torch.cat((img_projection, txt_projection), dim=0)
    # print(low_merge.shape)
    img_projection=torch.flatten(img_projection, start_dim=1)
    num = img_projection.shape[0]
    low_img_matrix = img_projection.repeat(num, 1, 1)
    txt_projection=torch.flatten(txt_projection, start_dim=1)
    num1 = txt_projection.shape[0]
    low_txt_matrix = txt_projection.repeat(num1, 1, 1)
    low_txt_matrix0 = txt_projection.repeat(num, 1, 1)
    low_img_matrix0 = img_projection.repeat(num1, 1, 1)
    low_img_dis = torch.norm(low_img_matrix - low_img_matrix.transpose(0, 1), dim=-1)
    low_txt_dis = torch.norm(low_txt_matrix - low_txt_matrix.transpose(0, 1), dim=-1)
    low_merge=torch.flatten(low_merge, start_dim=1)
    num2=low_merge.shape[0]
    low_merge_matrix=low_merge.repeat(num2, 1, 1)
    low_merge_dis = torch.norm(low_merge_matrix - low_merge_matrix.transpose(0, 1), dim=-1) 
    low_cross_dis = torch.norm(low_txt_matrix0 - low_img_matrix0.transpose(0, 1), dim=-1)
    img_corr = torch.mean(
    pearsonr(
        img_dis,
        low_img_dis,
    )[:, 0])
    txt_corr = torch.mean(
    pearsonr(
        txt_dis,
        low_txt_dis,
    )[:, 0])
    cross_corr = torch.mean(
    pearsonr(
        cross_dis,
        low_cross_dis,
    )[:, 0])
    all_corr= torch.mean(
    pearsonr(
        merge_dis,
        low_merge_dis,
    )[:, 0])
    return -img_corr, -txt_corr, -cross_corr, -all_corr, merge_dis, img_dis, txt_dis, cross_dis, low_merge_dis, low_img_dis, low_txt_dis, low_cross_dis

def pearson_loss(low_dis, high_dis):
    corr=torch.mean(
    pearsonr(
        high_dis,
        low_dis,
    )[:, 0])
    return -corr
def global_feature_loss(x_embeddings, to_x):
        # embedding_to, embedding_from = torch.chunk(x_embeddings, 2, dim=1)
        embedding_to = x_embeddings
        # print(embedding_to.shape)
        
        to_x = torch.flatten(to_x, start_dim=1)
#         print(to_x.shape)
        num = to_x.shape[0]
        high_matrix = to_x.repeat(num, 1, 1)
        print(high_matrix.shape)
        low_matrix = embedding_to.repeat(num, 1, 1)
#         print(low_matrix.shape)

        high_dis = torch.norm(high_matrix - high_matrix.transpose(0, 1), dim=-1)
        low_dis = torch.norm(low_matrix - low_matrix.transpose(0, 1), dim=-1)
        print(high_dis.shape)
        print(low_dis.shape)
        corr = torch.mean(
            pearsonr(
                high_dis,
                low_dis,
            )[:, 0])
        return -corr

In [102]:
import clip
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32")
clip_model.cuda().eval()
input_resolution = clip_model.visual.input_resolution
context_length = clip_model.context_length
vocab_size = clip_model.vocab_size



In [7]:
def retrieve_ball_search_C(x, K):
    """ find K nearest neighbours of data among D """
    x_norm=np.linalg.norm(x)
    x=x.reshape((1,-1))
    x=x/x_norm
    result=retrieve_ball.query(x, k=K)
    idx=result[1]
    idx=idx[0]
    return idx

In [105]:
from PIL import Image
text_tokens = clip.tokenize(all_cats).cuda()
with torch.no_grad():
    text_features = clip_model.encode_text(text_tokens).float()
text_features /= text_features.norm(dim=-1, keepdim=True)

In [ ]:
def cosDis(x,D):
    return 1-np.dot(D,x)/(np.linalg.norm(D,axis=1)*np.linalg.norm(x))
def cosD(x,y):
    return 1-(np.dot(x,y)/np.linalg.norm(y))/np.linalg.norm(x)
def EuDis(x,D):
    return np.linalg.norm(x-D,axis=1)
def EuD(x,y):
    return np.linalg.norm(x-y)
def knn_search_C(x, D, K):
    """ find K nearest neighbours of data among D """
    ndata = D.shape[0]
    K = K if K < ndata else ndata
    sqd=EuDis(x,D)
    # print(sqd.shape)
    idx = sqd.argsort() # sorting
    return idx[:K]
def cross_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk):
    T=0
    for i in range(low_tx_emb.shape[0]):
        low_knn=knn_search_C(low_tx_emb[i], low_im_emb, tk)
        high_knn=knn_search_C(high_tx_emb[i], high_im_emb, tk)
        N=high_im_emb.shape[0]
        high_ordering=knn_search_C(high_tx_emb[i], high_im_emb, N)
        error_array=np.setdiff1d(low_knn, high_knn)
        sum_r=0
        for j in range(error_array.shape[0]):
            rj=np.where(high_ordering==error_array[j])[0][0]
            sum_r=sum_r+(rj-tk)
            
        Ak=2/(N*tk*(2*N-3*tk-1))
        T=T+Ak*sum_r
    return 1-T
def img_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk):
    T=0
    for i in range(low_im_emb.shape[0]):
        low_knn=knn_search_C(low_im_emb[i], low_im_emb, tk)
        high_knn=knn_search_C(high_im_emb[i], high_im_emb, tk)
        N=high_im_emb.shape[0]
        high_ordering=knn_search_C(high_im_emb[i], high_im_emb, N)
        error_array=np.setdiff1d(low_knn, high_knn)
        sum_r=0
        for j in range(error_array.shape[0]):
            rj=np.where(high_ordering==error_array[j])[0][0]
            sum_r=sum_r+(rj-tk)
            
        Ak=2/(N*tk*(2*N-3*tk-1))
        T=T+Ak*sum_r
    return 1-T
def img_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk):
    T=0
    for i in range(low_im_emb.shape[0]):
        low_knn=knn_search_C(low_im_emb[i], low_im_emb, tk)
        high_knn=knn_search_C(high_im_emb[i], high_im_emb, tk)
        N=high_im_emb.shape[0]
#         high_ordering=knn_search_C(high_tx_emb[i], high_im_emb, N)
        low_ordering=knn_search_C(low_im_emb[i], low_im_emb, N)
        error_array=np.setdiff1d(high_knn, low_knn)
        sum_r=0
        for j in range(error_array.shape[0]):
            rj=np.where(low_ordering==error_array[j])[0][0]
            sum_r=sum_r+(rj-tk)
            
        Ak=2/(N*tk*(2*N-3*tk-1))
        T=T+Ak*sum_r
    return 1-T
def txt_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk):
    T=0
    for i in range(low_tx_emb.shape[0]):
        low_knn=knn_search_C(low_tx_emb[i], low_tx_emb, tk)
        high_knn=knn_search_C(high_tx_emb[i], high_tx_emb, tk)
        N=high_tx_emb.shape[0]
        high_ordering=knn_search_C(high_tx_emb[i], high_tx_emb, N)
        error_array=np.setdiff1d(low_knn, high_knn)
        sum_r=0
        for j in range(error_array.shape[0]):
            rj=np.where(high_ordering==error_array[j])[0][0]
            sum_r=sum_r+(rj-tk)
            
        Ak=2/(N*tk*(2*N-3*tk-1))
        T=T+Ak*sum_r
    return 1-T
def txt_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk):
    T=0
    for i in range(low_tx_emb.shape[0]):
        low_knn=knn_search_C(low_tx_emb[i], low_tx_emb, tk)
        high_knn=knn_search_C(high_tx_emb[i], high_tx_emb, tk)
        N=high_tx_emb.shape[0]
#         high_ordering=knn_search_C(high_tx_emb[i], high_im_emb, N)
        low_ordering=knn_search_C(low_tx_emb[i], low_tx_emb, N)
        error_array=np.setdiff1d(high_knn, low_knn)
        sum_r=0
        for j in range(error_array.shape[0]):
            rj=np.where(low_ordering==error_array[j])[0][0]
            sum_r=sum_r+(rj-tk)
            
        Ak=2/(N*tk*(2*N-3*tk-1))
        T=T+Ak*sum_r
    return 1-T
def cross_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk):
    T=0
    for i in range(low_tx_emb.shape[0]):
        low_knn=knn_search_C(low_tx_emb[i], low_im_emb, tk)
        high_knn=knn_search_C(high_tx_emb[i], high_im_emb, tk)
        N=high_im_emb.shape[0]
#         high_ordering=knn_search_C(high_tx_emb[i], high_im_emb, N)
        low_ordering=knn_search_C(low_tx_emb[i], low_im_emb, N)
        error_array=np.setdiff1d(high_knn, low_knn)
        sum_r=0
        for j in range(error_array.shape[0]):
            rj=np.where(low_ordering==error_array[j])[0][0]
            sum_r=sum_r+(rj-tk)
            
        Ak=2/(N*tk*(2*N-3*tk-1))
        T=T+Ak*sum_r
    return 1-T

In [ ]:
import random
import time
import umap
from PIL import Image
# umap_fit = umap.UMAP(n_neighbors=20)
# KDCM_list=[]
# PDCM_list=[]
# DCM_list=[]
# tsne_list=[]
# PCA_list=[]
# MDS_list=[]
# Isomap_list=[]
# NDCM_list=[]
# MKPE_list=[]
# UMAP_list=[]
# KDCM_list1=[]
# PDCM_list1=[]
# DCM_list1=[]
# tsne_list1=[]
# PCA_list1=[]
# MDS_list1=[]
# Isomap_list1=[]
# NDCM_list1=[]
# MKPE_list1=[]
# UMAP_list1=[]
# KDCM_list2=[]
# PDCM_list2=[]
# DCM_list2=[]
# tsne_list2=[]
# PCA_list2=[]
# MDS_list2=[]
# Isomap_list2=[]
# NDCM_list2=[]
# MKPE_list2=[]
# UMAP_list2=[]
# KDCM_list3=[]
# PDCM_list3=[]
# DCM_list3=[]
# tsne_list3=[]
# PCA_list3=[]
# MDS_list3=[]
# Isomap_list3=[]
# NDCM_list3=[]
# MKPE_list3=[]
# UMAP_list3=[]
# KDCM_list4=[]
# PDCM_list4=[]
# DCM_list4=[]
# tsne_list4=[]
# PCA_list4=[]
# MDS_list4=[]
# Isomap_list4=[]
# NDCM_list4=[]
# MKPE_list4=[]
# UMAP_list4=[]
# KDCM_list5=[]
# PDCM_list5=[]
# DCM_list5=[]
# tsne_list5=[]
# PCA_list5=[]
# MDS_list5=[]
# Isomap_list5=[]
# NDCM_list5=[]
# MKPE_list5=[]
# UMAP_list5=[]
# KDCM_list6=[]
# PDCM_list6=[]
# DCM_list6=[]
# tsne_list6=[]
# PCA_list6=[]
# MDS_list6=[]
# Isomap_list6=[]
# NDCM_list6=[]
# MKPE_list6=[]
# UMAP_list6=[]
#118287
tk=20
bsize=1000
multiply=1
# for r in range(1):
#     print(r)
ids=[]
#     for i in range(bsize*multiply):
for i in range(118287):
#         seed=(int)(len(all_path_list)*random.random())
    seed=i
    ids.append(seed)
#     from PIL import Image
text_tokens = clip.tokenize(all_cats).cuda()
with torch.no_grad():
    text_features = clip_model.encode_text(text_tokens).float()
text_features /= text_features.norm(dim=-1, keepdim=True)
print("loading data")


im_feature_list=[]
for m in range(multiply):
    print(m)
    imgs=[]
    for i in range(bsize*m,min(bsize*(m+1),118287)):
        image=Image.open("D:/Test_Disen/Backend/train2017/"+all_path_list[ids[i]])
        imgs.append(preprocess(image))
#         image_input = torch.tensor(np.stack(imgs[bsize*m:bsize*(m+1)])).cuda()
    image_input = torch.tensor(np.stack(imgs)).cuda()
    with torch.no_grad():
        image_features = clip_model.encode_image(image_input).float()
    image_features /= image_features.norm(dim=-1, keepdim=True)
    im_features=image_features.cpu().numpy()
    im_feature_list.append(im_features)

im_features=np.concatenate(im_feature_list)
print(im_features.shape)
tx_features=text_features.cpu().numpy()
#     text_features=text_features.cpu().numpy()
print("doing PDCM")   
ProjTextModel = NeuralProj().to(device)
ProjImgModel = NeuralProj().to(device)






#There is a bug when only one text anchor_point

#     dataset = image_textanchor_dataset(image_features, text_features)
dataset = image_textanchor_dataset1(im_features, tx_features, multiply, bsize)
train_dataloader = DataLoader(dataset, batch_size = 1)
optimizer = optim.Adam([{"params": ProjTextModel.parameters()}, {"params": ProjImgModel.parameters()}], lr=1e-5,
                    betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
Cos = nn.CosineSimilarity()
# ep=500
ep=1000
start0=time.time()
for epoch in range(ep):
#         print(epoch)
    # print(epoch)
    c=0
    ImgProjList=[]
#         TextProjList=[]
    for batch in train_dataloader:
        optimizer.zero_grad()
        images,texts= batch 
        images= images.to(device)
        texts = texts.to(device)
        TextProj=ProjTextModel(texts)
        ImgProj=ProjImgModel(images)
#             print(images.shape)
#             print(ImgProj.shape)

        loss=0
        img_loss, txt_loss, cross_loss, all_loss, _, _, _, cross_dis, _, _, _, low_cross_dis=data_context_map(images[0],texts[0],ImgProj[0],TextProj[0])
#             print(low_cross_dis[knns_list[0],[0]].reshape((-1,1)).shape)
        if epoch>100:
            loss=loss+10*all_loss+2*cross_loss#+img_loss+txt_loss+2*cross_loss
        else:
            loss=loss+img_loss
#             loss=loss+10*all_loss
        loss0=loss
#             if epoch>400:
        if epoch>800:
            for i in range(text_features.shape[0]):
                if c<multiply-1:
                    loss=loss+0.05*torch.norm(rank_loss(images[0], texts[0,i], ImgProj[0], TextProj[0,i], bsize))
                else:
                    loss=loss+0.05*torch.norm(rank_loss(images[0], texts[0,i], ImgProj[0], TextProj[0,i], 287))
#                     loss=loss+0.2*torch.norm(rank_loss(images[0], texts[0,i], ImgProj[0], TextProj[0,i], 500))
        rank_gap=loss-loss0
        # if epoch==ep-1:
        ImgProjList.append(ImgProj[0].detach().cpu().numpy())

        loss.backward()
        optimizer.step()
        c=c+1
        if c%10==0:
            print(c)
#     print("epoch")
#     print(epoch)
#     mmd_distance_metric = compute_mmd(TextProj[0], ImgProj[0], sigma=1.0)
end0=time.time()
lapse0=end0-start0
print("time0")
print(lapse0)

#     img_array=ImgProj[0].detach().cpu().numpy()
img_array=np.concatenate(ImgProjList)
print(img_array.shape)
txt_array=TextProj[0].detach().cpu().numpy()
pre1_t=np.concatenate([img_array, txt_array])
x_min, x_max = pre1_t.min(0), pre1_t.max(0)
pre1_norm0 = (pre1_t - x_min) / (x_max - x_min)

del images
torch.cuda.empty_cache()
del texts
torch.cuda.empty_cache()
del TextProj
torch.cuda.empty_cache()
del ImgProj
torch.cuda.empty_cache()




#     dissimilarity=merge_matrix(im_features, tx_features)

#     pre1_t=mds1.fit_transform(dissimilarity)
#     x_min, x_max = pre1_t.min(0), pre1_t.max(0)
#     pre1_norm1 = (pre1_t - x_min) / (x_max - x_min)
#     low_im_emb=pre1_norm1[:multiply*bsize]    
#     low_tx_emb=pre1_norm1[multiply*bsize:]

#     for batch in train_dataloader:
#         # optimizer.zero_grad()
#         images,texts= batch 
#         images= images.to(device)
#         texts = texts.to(device)
#         TextProj=ProjTextModel(texts)
#         ImgProj=ProjImgModel(images)
#     _, _, _, _, merge_dis, img_dis, txt_dis, cross_dis, _, _, _, _=data_context_map(images[0], texts[0], ImgProj[0], TextProj[0])
#     dissimilarity=merge_dis.cpu().numpy()


#     pre1_t=mds1.fit_transform(dissimilarity)
#     x_min, x_max = pre1_t.min(0), pre1_t.max(0)
#     pre1_norm1 = (pre1_t - x_min) / (x_max - x_min)

#     pre1_t=mds2.fit_transform(dissimilarity)
#     x_min, x_max = pre1_t.min(0), pre1_t.max(0)
#     pre1_normN = (pre1_t - x_min) / (x_max - x_min)

# #     pre1_t=MKPE(img_dis, txt_dis, cross_dis)
# #     x_min, x_max = pre1_t.min(0), pre1_t.max(0)
# #     pre1_normM = (pre1_t - x_min) / (x_max - x_min)

# #     all_ems=np.concatenate([image_features.cpu().numpy(), text_features.cpu().numpy()])
#     print("doing t-SNE")
#     all_ems=np.concatenate([im_features, tx_features])   
# #     tsne = manifold.TSNE(n_components=2, init='pca', random_state=501, metric='cosine', perplexity=5)
#     pre1_t=tsne.fit_transform(all_ems)
#     x_min, x_max = pre1_t.min(0), pre1_t.max(0)
#     pre1_norm2 = (pre1_t - x_min) / (x_max - x_min)
#     pre1_t=umap_fit.fit_transform(all_ems)
#     x_min, x_max = pre1_t.min(0), pre1_t.max(0)
#     pre1_normU = (pre1_t - x_min) / (x_max - x_min)
# #     pca = PCA(n_components=2)
#     pre1_t=pca.fit_transform(all_ems)
#     x_min, x_max = pre1_t.min(0), pre1_t.max(0)
#     pre1_norm3 = (pre1_t - x_min) / (x_max - x_min)
# #     mds = MDS(n_components=2)
#     pre1_t=mds.fit_transform(all_ems)
#     x_min, x_max = pre1_t.min(0), pre1_t.max(0)
#     pre1_norm4 = (pre1_t - x_min) / (x_max - x_min)
# #     iso = Isomap(n_components=2)
#     pre1_t=iso.fit_transform(all_ems)
#     x_min, x_max = pre1_t.min(0), pre1_t.max(0)
#     pre1_norm5 = (pre1_t - x_min) / (x_max - x_min)

#     high_im_emb=image_features.cpu().numpy()
#     high_tx_emb=text_features.cpu().numpy()
high_im_emb=im_features
high_tx_emb=tx_features
low_im_emb=pre1_norm0[:multiply*bsize]    
low_tx_emb=pre1_norm0[multiply*bsize:]
print("PDCM")
#     acc=cross_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc1=cross_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc2=img_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc3=img_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc4=txt_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc5=txt_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     PDCM_list.append(acc)
#     PDCM_list1.append(acc1)
#     PDCM_list2.append(acc2)
#     PDCM_list3.append(acc3)
#     PDCM_list4.append(acc4)
#     PDCM_list5.append(acc5)
#     print(acc)
#     print(acc1)
#     print(acc2)
#     print(acc3)
#     print(acc4)
#     print(acc5)
#     print("mmd_distance")
#     print(mmd_distance_metric)
#     print("time")
print(lapse0)
#     PDCM_list6.append(mmd_distance_metric)
#     low_im_emb=pre1_norm01[:multiply*bsize]    
#     low_tx_emb=pre1_norm01[multiply*bsize:]
#     print("KDCM")
#     acc=cross_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc1=cross_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc2=img_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc3=img_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc4=txt_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc5=txt_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     KDCM_list.append(acc)
#     KDCM_list1.append(acc1)
#     KDCM_list2.append(acc2)
#     KDCM_list3.append(acc3)
#     KDCM_list4.append(acc4)
#     KDCM_list5.append(acc5)
#     print(acc)
#     print(acc1)
#     print(acc2)
#     print(acc3)
#     print(acc4)
#     print(acc5)

#     low_im_emb=pre1_norm02[:multiply*bsize]    
#     low_tx_emb=pre1_norm02[multiply*bsize:]
#     print("KDCM2")
#     acc=cross_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc1=cross_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc2=img_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc3=img_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc4=txt_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc5=txt_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     DCM_list.append(acc)
#     DCM_list1.append(acc1)
#     DCM_list2.append(acc2)
#     DCM_list3.append(acc3)
#     DCM_list4.append(acc4)
#     DCM_list5.append(acc5)
#     print(acc)
#     print(acc1)
#     print(acc2)
#     print(acc3)
#     print(acc4)
#     print(acc5)


#     low_im_emb=pre1_norm03[:multiply*bsize]    
#     low_tx_emb=pre1_norm03[multiply*bsize:]
# #     mmd_NDCM = compute_mmd(torch.Tensor(low_tx_emb), torch.Tensor(low_im_emb), sigma=1.0)
#     print("KDCM3")
#     acc=cross_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc1=cross_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc2=img_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc3=img_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc4=txt_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc5=txt_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     NDCM_list.append(acc)
#     NDCM_list1.append(acc1)
#     NDCM_list2.append(acc2)
#     NDCM_list3.append(acc3)
#     NDCM_list4.append(acc4)
#     NDCM_list5.append(acc5)
#     NDCM_list6.append(mmd_NDCM)
#     print(acc)
#     print(acc1)
#     print(acc2)
#     print(acc3)
#     print(acc4)
#     print(acc5)

#     low_im_emb=pre1_norm04[:multiply*bsize]    
#     low_tx_emb=pre1_norm04[multiply*bsize:]
# #     mmd_tsne = compute_mmd(torch.Tensor(low_tx_emb), torch.Tensor(low_im_emb), sigma=1.0)
#     print("KDCM4")
#     acc=cross_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc1=cross_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc2=img_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc3=img_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc4=txt_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc5=txt_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     tsne_list.append(acc)
#     tsne_list1.append(acc1)
#     tsne_list2.append(acc2)
#     tsne_list3.append(acc3)
#     tsne_list4.append(acc4)
#     tsne_list5.append(acc5)
#     print(acc)
#     print(acc1)
#     print(acc2)
#     print(acc3)
#     print(acc4)
#     print(acc5)
#     print("time")
#     print(lapse1)
#     print("mmd_distance1")
#     print(mmd_distance_metric1)

#     low_im_emb=pre1_norm1[:multiply*bsize]    
#     low_tx_emb=pre1_norm1[multiply*bsize:]
#     mmd_DCM = compute_mmd(torch.Tensor(low_tx_emb), torch.Tensor(low_im_emb), sigma=1.0)
#     print("DCM")
#     acc=cross_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc1=cross_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc2=img_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc3=img_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc4=txt_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc5=txt_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     DCM_list.append(acc)
#     DCM_list1.append(acc1)
#     DCM_list2.append(acc2)
#     DCM_list3.append(acc3)
#     DCM_list4.append(acc4)
#     DCM_list5.append(acc5)
#     DCM_list6.append(mmd_DCM)
#     print(acc)
#     print(acc1)
#     print(acc2)
#     print(acc3)
#     print(acc4)
#     print(acc5)
#     print("mmd_distance")
#     print(mmd_DCM)

#     low_im_emb=pre1_normN[:multiply*bsize]    
#     low_tx_emb=pre1_normN[multiply*bsize:]
#     mmd_NDCM = compute_mmd(torch.Tensor(low_tx_emb), torch.Tensor(low_im_emb), sigma=1.0)
#     print("NDCM")
#     acc=cross_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc1=cross_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc2=img_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc3=img_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc4=txt_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc5=txt_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     NDCM_list.append(acc)
#     NDCM_list1.append(acc1)
#     NDCM_list2.append(acc2)
#     NDCM_list3.append(acc3)
#     NDCM_list4.append(acc4)
#     NDCM_list5.append(acc5)
#     NDCM_list6.append(mmd_NDCM)
#     print(acc)
#     print(acc1)
#     print(acc2)
#     print(acc3)
#     print(acc4)
#     print(acc5)
#     print("mmd_distance")
#     print(mmd_NDCM)

#     low_im_emb=pre1_normM[:bsize]    
#     low_tx_emb=pre1_normM[bsize:]
#     print("MKPE")
#     acc=cross_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc1=cross_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc2=img_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc3=img_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc4=txt_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc5=txt_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     MKPE_list.append(acc)
#     MKPE_list1.append(acc1)
#     MKPE_list2.append(acc2)
#     MKPE_list3.append(acc3)
#     MKPE_list4.append(acc4)
#     MKPE_list5.append(acc5)
#     print(acc)
#     print(acc1)
#     print(acc2)
#     print(acc3)
#     print(acc4)
#     print(acc5)


#     low_im_emb=pre1_norm2[:multiply*bsize]    
#     low_tx_emb=pre1_norm2[multiply*bsize:]
#     mmd_tsne = compute_mmd(torch.Tensor(low_tx_emb), torch.Tensor(low_im_emb), sigma=1.0)
#     print("t-SNE")
#     acc=cross_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc1=cross_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc2=img_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc3=img_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc4=txt_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc5=txt_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     tsne_list.append(acc)
#     tsne_list1.append(acc1)
#     tsne_list2.append(acc2)
#     tsne_list3.append(acc3)
#     tsne_list4.append(acc4)
#     tsne_list5.append(acc5)
#     tsne_list6.append(mmd_tsne)
#     print(acc)
#     print(acc1)
#     print(acc2)
#     print(acc3)
#     print(acc4)
#     print(acc5)
#     print("mmd_distance")
#     print(mmd_tsne)

#     low_im_emb=pre1_normU[:multiply*bsize]    
#     low_tx_emb=pre1_normU[multiply*bsize:]
#     mmd_umap = compute_mmd(torch.Tensor(low_tx_emb), torch.Tensor(low_im_emb), sigma=1.0)
#     print("UMAP")
#     acc=cross_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc1=cross_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc2=img_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc3=img_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc4=txt_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc5=txt_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     UMAP_list.append(acc)
#     UMAP_list1.append(acc1)
#     UMAP_list2.append(acc2)
#     UMAP_list3.append(acc3)
#     UMAP_list4.append(acc4)
#     UMAP_list5.append(acc5)
#     UMAP_list6.append(mmd_umap)
#     print(acc)
#     print(acc1)
#     print(acc2)
#     print(acc3)
#     print(acc4)
#     print(acc5)
#     print("mmd_distance")
#     print(mmd_umap)    

#     low_im_emb=pre1_norm3[:multiply*bsize]    
#     low_tx_emb=pre1_norm3[multiply*bsize:]
#     mmd_pca = compute_mmd(torch.Tensor(low_tx_emb), torch.Tensor(low_im_emb), sigma=1.0)
#     print("PCA")
#     acc=cross_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc1=cross_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc2=img_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc3=img_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc4=txt_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc5=txt_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     PCA_list.append(acc)
#     PCA_list1.append(acc1)
#     PCA_list2.append(acc2)
#     PCA_list3.append(acc3)
#     PCA_list4.append(acc4)
#     PCA_list5.append(acc5)
#     PCA_list6.append(mmd_pca)
#     print(acc)
#     print(acc1)
#     print(acc2)
#     print(acc3)
#     print(acc4)
#     print(acc5)
#     print("mmd_distance")
#     print(mmd_pca)  


#     low_im_emb=pre1_norm4[:multiply*bsize]    
#     low_tx_emb=pre1_norm4[multiply*bsize:]
#     mmd_mds = compute_mmd(torch.Tensor(low_tx_emb), torch.Tensor(low_im_emb), sigma=1.0)
#     print("MDS")
#     acc=cross_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc1=cross_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc2=img_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc3=img_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc4=txt_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc5=txt_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     MDS_list.append(acc)
#     MDS_list1.append(acc1)
#     MDS_list2.append(acc2)
#     MDS_list3.append(acc3)
#     MDS_list4.append(acc4)
#     MDS_list5.append(acc5)
#     MDS_list6.append(mmd_mds)
#     print(acc)
#     print(acc1)
#     print(acc2)
#     print(acc3)
#     print(acc4)
#     print(acc5)
#     print("mmd_distance")
#     print(mmd_mds)


#     low_im_emb=pre1_norm5[:multiply*bsize]    
#     low_tx_emb=pre1_norm5[multiply*bsize:]
#     mmd_isomap = compute_mmd(torch.Tensor(low_tx_emb), torch.Tensor(low_im_emb), sigma=1.0)
#     print("Isomap")
#     acc=cross_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc1=cross_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc2=img_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc3=img_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc4=txt_trust(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     acc5=txt_continuity(high_im_emb, high_tx_emb, low_im_emb, low_tx_emb, tk)
#     Isomap_list.append(acc)
#     Isomap_list1.append(acc1)
#     Isomap_list2.append(acc2)
#     Isomap_list3.append(acc3)
#     Isomap_list4.append(acc4)
#     Isomap_list5.append(acc5)
#     Isomap_list6.append(mmd_isomap)
#     print(acc)
#     print(acc1)
#     print(acc2)
#     print(acc3)
#     print(acc4)
#     print(acc5)
#     print("mmd_distance")
#     print(mmd_isomap)

0
loading data
(1000, 512)
doing PDCM
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267


C:\Users\user\AppData\Local\Temp\ipykernel_8392\640338840.py:32: RuntimeWarning: invalid value encountered in double_scalars
  h_i = beta * np.sum(d_i * p_i) / s + np.log(s)
C:\Users\user\AppData\Local\Temp\ipykernel_8392\640338840.py:32: RuntimeWarning: divide by zero encountered in log
  h_i = beta * np.sum(d_i * p_i) / s + np.log(s)
C:\Users\user\AppData\Local\Temp\ipykernel_8392\640338840.py:35: RuntimeWarning: invalid value encountered in divide
  p_i /= s


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


993
994
995
996
997
998
999
time0
39.71283555030823
doing KDCM
0


C:\Users\user\AppData\Local\Temp\ipykernel_8392\640338840.py:32: RuntimeWarning: invalid value encountered in double_scalars
  h_i = beta * np.sum(d_i * p_i) / s + np.log(s)
C:\Users\user\AppData\Local\Temp\ipykernel_8392\640338840.py:32: RuntimeWarning: divide by zero encountered in log
  h_i = beta * np.sum(d_i * p_i) / s + np.log(s)
C:\Users\user\AppData\Local\Temp\ipykernel_8392\640338840.py:35: RuntimeWarning: invalid value encountered in divide
  p_i /= s


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
